In [5]:
import numpy as np
from scipy.optimize import differential_evolution

# 定义参数，这是情况1，你们可以吧2-6通过更改参数来测试
p1, p2 = 0.1, 0.1  # 零配件 1 和 2 的次品率
Cd1, Cd2 = 2, 3  # 零配件 1 和 2 的检测成本
Cp1, Cp2 = 4, 18  # 零配件 1 和 2 的采购单价
Ca, Cd = 6, 3  # 成品装配成本和检测成本
Ct, Cr = 5, 6  # 拆解费用和调换损失
n1, n2 = 100, 100  # 零配件数量
nf = 90  # 成品数量

# 成品次品率计算函数
def calculate_defect_rate(x1, x2):
    PAa = (1 - x1) * p1 + (1 - x2) * p2 - (1 - x1) * (1 - x2) * p1 * p2
    pf = 1 - (1 - PAa) ** 2
    return pf

# 成本计算函数
def total_cost(x):
    x1, x2, xf, xt = x  # 决策变量
    pf = calculate_defect_rate(x1, x2)
    
    # 零配件检测成本
    C_零配件检测 = x1 * Cd1 * n1 + x2 * Cd2 * n2
    
    # 装配成本
    C_装配 = Ca * nf
    
    # 成品检测成本
    C_成品检测 = xf * Cd * nf
    
    # 拆解和调换成本
    C_拆解 = xt * Ct * pf * nf
    C_调换 = (1 - xt) * Cr * pf * nf
    
    # 总成本
    total_cost = C_零配件检测 + C_装配 + C_成品检测 + C_拆解 + C_调换
    return total_cost

# 使用遗传算法优化
bounds = [(0, 1), (0, 1), (0, 1), (0, 1)]  # 决策变量的范围

result = differential_evolution(total_cost, bounds, strategy='best1bin', maxiter=1000, popsize=15, tol=0.01)
optimal_solution = result.x  # 最优解
optimal_cost = result.fun  # 最优总成本

print(f"最优解: {optimal_solution}")
print(f"最优总成本: {optimal_cost:.2f}")


最优解: [0. 0. 0. 1.]
最优总成本: 694.75


In [ ]:
import creator

creator.create("FitnessMin",base.Fitness, weights=(-1.0,))
